Copied from the awesome notebook "Simple MLP".
Just add simple plot to show kaggle novices like me why it is overfitting. 

Score of the version 1 : 10609.091 (slightly better than the original 10606.782)

.................... 2 : 7340.576

.................... 3 : 7521.166

.................... 6 : 6567.191

.................... 7 : Pending

## Kudos: 🔥🔥🔥

[jane street with keras nn overfit](https://www.kaggle.com/code1110/jane-street-with-keras-nn-overfit/)

[OWN Jane Street with Keras NN](https://www.kaggle.com/tarlannazarov/own-jane-street-with-keras-nn)



This notebook overfits after almost 10 epochs, but it's good to make some experimentation.

You can add a validation set, experiment with things like Purged Time Series...


## If you change the following your score will changes a lot in some case:

1/ Seed : I changed from 1111 to 73 and 1112, and the score changes, so having a good seed guess will work for your advantage.

2/ th : it's the threshold, parameter, and the best is 0.503 (the detail are below).

3/ optimizers : Comparing the two notebook mentionned above, RectifiedAdam really gives better results.

4/ epochs : starts over fitting around 15 epochs, so a higher epoch number will give you a better score on the public dataset, but probably won't generilize well. If you want to use a high number of epochs, and a lot of parameters you should have a validation dataset to watch out for overfitting.

5/ hidden_units : I have increased the hidden units number, but the score went down.

6/ date :  date > 85 because the data changes after around 4 months, and here we can make two models, one for the first 4 months, and another one for the rest, after that we stack them together.

7/ feature_0 :  1 for buy and -1 for sell (Bid/Ask), and weight for trading volume.  
=> Buy less, Sell more, so you increase the treshold for buy orders to 0.503 while for sell order reduce the threshold to 0.498, and we can drop this feature before training the model, and change it with other instructions in the prediction phase.

### P.S.
Those were a couple of my observations, and some of the comments that I have found usefull in the comment section on the above mentionned notebooks.

Please feel free to comment, and if You feel there is an added value of this NoteBook, consider to vote it up =)

# Libs

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
import gc

from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa

# Data Import and Processing

In [ ]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')

train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

In [ ]:
train.fillna(train.mean(),inplace=True)
features = [c for c in train.columns if 'feature' in c]

train['action'] = ((train['resp'].values) > 0).astype(int)
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']


X_train = train.loc[:, train.columns.str.contains('feature')]
#y_train = (train.loc[:, 'action'])

# resp_1 > 0 resp_2 > 0 ...
y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

# Model

In [ ]:
# from the comments https://www.kaggle.com/tarlannazarov/own-jane-street-with-keras-nn
# 1111 gives the best result
#  I tried other seed, and got high score that is more than 8000. 
# I have tried a couple of random seeds and the worst only gives 5000 score. 
SEED = 1112
np.random.seed(SEED)

#  initial parameters from Keras Tuner bayesian optimization

# fit
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
    
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate), # RectifiedAdam Optimizer (known to be robust to the choice in learning rate)
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    ) 

    return model

epochs = 2000
batch_size = 4096
hidden_units = [160, 160, 160]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
#Label Smoothing is a regularization technique that introduces noise for the labels. 
#This accounts for the fact that datasets may have mistakes in them, so maximizing the likelihood of directly can be harmful.
#Assume for a small constant e, the training set label y is correct with probability 1 - e and incorrect otherwise. 
#Label Smoothing regularizes a model based on a softmax with k output values by replacing the hard 0 and 1 classification targets 
#with e/k-1 targets of and 1 - e respectively.
learning_rate = 1e-5
#normally the model training with a batch size of 4096 and learning rate 1e-3 starts to overfit 
#on the train set after only 10 epochs. 

tf.keras.backend.clear_session()
tf.random.set_seed(SEED)

from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}-{val_AUC:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_AUC', verbose=1, save_best_only=True,mode='max')

clf = create_mlp(len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate)
history = clf.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0,validation_split=0.2,callbacks=[checkpoint])

auc = history.history['AUC']
val_auc = history.history['val_AUC']
loss_score = history.history['loss']
val_loss_score = history.history['val_loss']

history_df = pd.DataFrame({'AUC':auc,'val_AUC':val_auc,'loss':loss_score,'val_loss':val_loss_score})

history_df.to_csv('history_df.csv')

# save model
clf.save(f'model_v2.h5')

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(history.history['AUC'])
plt.plot(history.history['val_AUC'])
plt.legend(['train', 'test'], loc='upper left') 
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.title('model accuracy')
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'test'], loc='upper left') 
plt.ylabel('loss')
plt.xlabel('epoch')
plt.title('model loss')
plt.show()

# Prediction

In [ ]:
th = 0.503 # https://www.kaggle.com/gkoundry/the-most-important-model-parameter
# This parameter controls the ratio of ones and zeros in the prediction. If the market is going up, then you will want to predict more ones than zeros.
# However it's a little more complicated than this as you need to take the weights into consideration and not just the direction of the whole market:$
# In the training data the overall market is going up, but the weighted returns are trending downwards so you would want to predict fewer ones for this time period.
# => It's the threshold, and yes it's an intuitive experience. The model predicts probabilities of resp between 0 and 1, 
# compares its median to the th, and then assigns 0 or 1. 

f_mean = np.mean(train[features[1:]].values,axis=0)

import janestreet
env = janestreet.make_env()

for (test_df, pred_df) in tqdm(env.iter_test()): 
    if test_df['weight'].item() > 0:
        
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            # np.nan_to_num : Replace NaN with zero and infinity with large finite numbers
            # np.isnan : return a boolean list
            
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
            
        pred = np.median(clf(x_tt))
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
        
    else:
        pred_df.action = 0
        
    env.predict(pred_df)